### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# Punto 1

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import random

In [2]:
from sklearn.datasets import fetch_20newsgroups

# Cargamos solo el set de entrenamiento
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

texts = newsgroups_train.data
labels = newsgroups_train.target
label_names = newsgroups_train.target_names


In [3]:
import pandas as pd

df = pd.DataFrame({
    'text': texts,
    'label': labels
})
df['label_name'] = df['label'].apply(lambda x: label_names[x])
df.head()


,text,label,label_name
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space


In [4]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, min_df=5)
X = vectorizer.fit_transform(df['text'])
X.shape  # para ver dimensiones de la matriz resultante


(11314, 17797)

In [5]:
random.seed(42)
sample_indices = random.sample(range(len(df)), 5)
sample_indices  #indices elegidos

[10476, 1824, 409, 4506, 4012]

In [6]:
similarities = cosine_similarity(X[sample_indices], X)
similarities.shape  # (5, total de documentos)

(5, 11314)

In [7]:
for i, idx in enumerate(sample_indices):
    print(f"\n Documento base #{i+1} (índice {idx})")
    print("Texto (truncado):", df['text'].iloc[idx][:300].replace("\n", " "), "...")
    print("Etiqueta:", df['label_name'].iloc[idx])

    sim_scores = similarities[i]
    top5_idx = sim_scores.argsort()[::-1][1:6]  # Excluye el propio doc

    print("\n Documentos más similares:")
    for rank, sim_idx in enumerate(top5_idx, 1):
        print(f"\n#{rank} (índice {sim_idx}) - Similaridad: {sim_scores[sim_idx]:.4f}")
        print("Texto (truncado):", df['text'].iloc[sim_idx][:300].replace("\n", " "), "...")
        print("Etiqueta:", df['label_name'].iloc[sim_idx])



 Documento base #1 (índice 10476)
Texto (truncado): This is a general question for US readers:  How extensive is the playoff coverage down there?  In Canada, it is almost impossible not to watch a series on TV (ie the only two series I have not had an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because I'm in the wrong time zo ...
Etiqueta: rec.sport.hockey

 Documentos más similares:

#1 (índice 5064) - Similaridad: 0.2273
Texto (truncado):  I only have one comment on this:  You call this a *classic* playoff year and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston fan and I even realize that Chicago-Detroit games are THE most exciting games to watch. ...
Etiqueta: rec.sport.hockey

#2 (índice 2645) - Similaridad: 0.1752
Texto (truncado):      These people were very silly.  Any team that gets to the World Series can win the World Series, and anybody who ever expects a sweep is crazy.  If you put the best team in baseball in the Series again

### Conclusión Parte 1: Vectorización y Similaridad entre Documentos

Utilizando `TfidfVectorizer` y la métrica de similaridad coseno, se evaluó la cercanía temática entre documentos del dataset 20 Newsgroups. Se seleccionaron cinco documentos al azar y se identificaron los cinco textos más similares para cada uno.

Los resultados muestran que, en la mayoría de los casos, los documentos más similares pertenecen a la misma categoría temática que el documento base, lo cual valida que la vectorización TF-IDF es efectiva para capturar patrones semánticos generales.

Además, cuando aparecen documentos de otras categorías, suelen estar relacionados, como por ejemplo temas de deportes distintos o tópicos técnicos similares. Esto demuestra que la representación vectorial basada en frecuencia de términos es útil para tareas de agrupamiento, clasificación o recomendación de textos.

En general, los niveles de similaridad obtenidos (entre 0.2 y 0.4 en los mejores casos) son esperables en este tipo de análisis con texto sin preprocesamiento profundo. Aun así, los resultados tienen sentido y reflejan una buena base para la siguiente etapa del desafío.

# Punto 2

In [8]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

In [9]:
# Usamos el mismo DataFrame df con las columnas 'text' y 'label'
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

In [10]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, min_df=5, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [11]:
model_mnb = MultinomialNB()
model_mnb.fit(X_train_vec, y_train)

MultinomialNB()

In [12]:
y_pred_mnb = model_mnb.predict(X_test_vec)
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')
print("F1-score macro (MultinomialNB):", round(f1_mnb, 4))

print("\nReporte de clasificación:\n")
print(classification_report(y_test, y_pred_mnb, target_names=label_names))

F1-score macro (MultinomialNB): 0.6983

Reporte de clasificación:

                          precision    recall  f1-score   support

             alt.atheism       0.87      0.35      0.50        96
           comp.graphics       0.69      0.70      0.69       117
 comp.os.ms-windows.misc       0.73      0.60      0.66       118
comp.sys.ibm.pc.hardware       0.62      0.68      0.65       118
   comp.sys.mac.hardware       0.79      0.70      0.75       115
          comp.windows.x       0.74      0.85      0.79       119
            misc.forsale       0.73      0.71      0.72       117
               rec.autos       0.80      0.76      0.78       119
         rec.motorcycles       0.85      0.77      0.81       120
      rec.sport.baseball       0.86      0.83      0.85       119
        rec.sport.hockey       0.56      0.93      0.70       120
               sci.crypt       0.71      0.78      0.74       119
         sci.electronics       0.78      0.71      0.74       118
        

In [13]:
model_cnb = ComplementNB()
model_cnb.fit(X_train_vec, y_train)

ComplementNB()

In [14]:
y_pred_cnb = model_cnb.predict(X_test_vec)
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')
print("F1-score macro (ComplementNB):", round(f1_cnb, 4))

print("\nReporte de clasificación:\n")
print(classification_report(y_test, y_pred_cnb, target_names=label_names))

F1-score macro (ComplementNB): 0.7335

Reporte de clasificación:

                          precision    recall  f1-score   support

             alt.atheism       0.39      0.55      0.46        96
           comp.graphics       0.77      0.69      0.73       117
 comp.os.ms-windows.misc       0.75      0.64      0.69       118
comp.sys.ibm.pc.hardware       0.61      0.69      0.65       118
   comp.sys.mac.hardware       0.83      0.71      0.77       115
          comp.windows.x       0.73      0.85      0.79       119
            misc.forsale       0.71      0.69      0.70       117
               rec.autos       0.82      0.78      0.80       119
         rec.motorcycles       0.91      0.80      0.85       120
      rec.sport.baseball       0.88      0.85      0.86       119
        rec.sport.hockey       0.87      0.92      0.89       120
               sci.crypt       0.86      0.80      0.83       119
         sci.electronics       0.78      0.70      0.74       118
         

### Conclusión Parte 2: Clasificación con Naive Bayes

Se entrenaron y evaluaron dos modelos de clasificación de texto sobre el dataset 20 Newsgroups utilizando vectorización TF-IDF:

- **Multinomial Naive Bayes (MNB)** alcanzó un f1-score macro de **0.6983**.
- **Complement Naive Bayes (CNB)** logró un mejor rendimiento, con un f1-score macro de **0.7335**.

El modelo ComplementNB mostró mejoras notables en varias clases, especialmente en categorías como `rec.sport.hockey`, `soc.religion.christian` y `sci.crypt`, lo cual es consistente con lo visto en clase, ya que este modelo suele funcionar mejor en datasets desbalanceados o con alta dispersión de palabras.

Ambos modelos lograron un buen desempeño general, pero CNB resultó más robusto, obteniendo mejor recall y precisión promedio en la mayoría de las categorías. Esto sugiere que es una mejor opción para este tipo de tareas de clasificación de texto multiclase sin necesidad de entrenamiento complejo.

La diferencia entre ambos modelos también resalta la importancia del ajuste de parámetros y la selección del tipo de Naïve Bayes en función de la naturaleza del dataset.

# Punto 3

In [15]:
# Transponemos la matriz para obtener término-documento
X_td = X_train_vec.transpose()
X_td.shape  # filas: palabras, columnas: documentos

(22580, 9051)

In [16]:
# Lista de términos (palabras) en el vocabulario
terms = vectorizer.get_feature_names_out()
terms[:10]  # muestra las primeras 10

array(['00', '00 00', '00 01', '00 10', '00 book', '00 new', '00 pm',
       '00 shipping', '000', '000 000'], dtype=object)

In [17]:
selected_words = ['jesus', 'glock', 'hockey', 'mac', 'clutch']

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

for word in selected_words:
    if word not in terms:
        print(f"La palabra '{word}' no está en el vocabulario.")
        continue

    idx = np.where(terms == word)[0][0]
    word_vector = X_td[idx]
    similarities = cosine_similarity(word_vector, X_td).flatten()

    # Obtener los 5 términos más similares (excluyendo el mismo)
    top_idx = similarities.argsort()[::-1][1:6]
    similar_words = [(terms[i], similarities[i]) for i in top_idx]

    print(f"\nPalabra base: '{word}'")
    for sim_word, score in similar_words:
        print(f"  Similar: {sim_word:<15} | Similaridad: {score:.4f}")



Palabra base: 'jesus'
  Similar: jesus christ    | Similaridad: 0.3406
  Similar: christ          | Similaridad: 0.3291
  Similar: jesus god       | Similaridad: 0.2921
  Similar: god             | Similaridad: 0.2844
  Similar: jesus did       | Similaridad: 0.2766

Palabra base: 'glock'
  Similar: revolver        | Similaridad: 0.4677
  Similar: jennings        | Similaridad: 0.4260
  Similar: misfire         | Similaridad: 0.4032
  Similar: intellectually  | Similaridad: 0.4001
  Similar: trigger         | Similaridad: 0.3673

Palabra base: 'hockey'
  Similar: hockey players  | Similaridad: 0.3128
  Similar: ncaa            | Similaridad: 0.2963
  Similar: nhl             | Similaridad: 0.2455
  Similar: college hockey  | Similaridad: 0.2293
  Similar: watch games     | Similaridad: 0.2149

Palabra base: 'mac'
  Similar: mac plus        | Similaridad: 0.2514
  Similar: drive mac       | Similaridad: 0.2383
  Similar: mac portable    | Similaridad: 0.2215
  Similar: mac iisi        